https://github.com/cloudacademy/using-the-azure-machine-learning-sdk/blob/main/03-Working_with_Data-SDK.ipynb

In [13]:
import azureml.core
from azureml.core import Workspace
import os

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.28.0 to work with dp-100-01


In [2]:
# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

automobiles - Default = False
azureml_globaldatasets - Default = False
workspaceblobstore - Default = True
workspacefilestore - Default = False


In [26]:
# Had trouble with path (can't do /data/data/diabetes.csv for some reason)
default_ds.upload_files(files=['diabetes.csv', 'diabetes2.csv'], # Upload the diabetes csv files in /data
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

Uploading an estimated of 2 files
Uploading diabetes2.csv
Uploaded diabetes2.csv, 1 files out of an estimated total of 2
Uploading diabetes.csv
Uploaded diabetes.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_455104f09d2f4059b506ef4492cfc35b

In [27]:
data_ref = default_ds.path('diabetes-data').as_download(path_on_compute='diabetes_data')
print(data_ref)

$AZUREML_DATAREFERENCE_d1b01b9c142645e496ff050cfc7f217d


In [28]:
type(data_ref)

azureml.data.data_reference.DataReference

## Load Dataset in tabular dataframe


In [30]:
from azureml.core import Dataset

# Get the default datastore
default_ds = ws.get_default_datastore()

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# Display the first 20 rows as a Pandas dataframe
tab_data_set.take(5).to_pandas_dataframe()


,PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1354778,0,171,80,34,23,43.509726,1.213191,21,0
1,1147438,8,92,93,47,36,21.240576,0.158365,23,0
2,1640031,7,115,47,52,35,41.511523,0.079019,23,0
3,1883350,9,103,78,25,304,29.582192,1.282870,43,1
4,1424119,1,85,59,27,35,42.604536,0.549542,22,0


## Unstructured data

In [31]:
#Create a file dataset from the path on the datastore (this may take a short while)
file_data_set = Dataset.File.from_files(path=(default_ds, 'diabetes-data/*.csv'))

# Get the files in the dataset
for file_path in file_data_set.to_path():
    print(file_path)

/diabetes.csv
/diabetes2.csv


## Upload to Dataset


In [32]:
# Register the tabular dataset
try:
    tab_data_set = tab_data_set.register(workspace=ws, 
                                        name='diabetes dataset',
                                        description='diabetes data',
                                        tags = {'format':'CSV'},
                                        create_new_version=True)
except Exception as ex:
    print(ex)

# Register the file dataset
try:
    file_data_set = file_data_set.register(workspace=ws,
                                            name='diabetes file dataset',
                                            description='diabetes files',
                                            tags = {'format':'CSV'},
                                            create_new_version=True)
except Exception as ex:
    print(ex)

print('Datasets registered')

Datasets registered


Can check inside Azure ML workspace under 'Datasets'

In [33]:
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

Datasets:
	 diabetes file dataset version 1
	 diabetes dataset version 1
	 automobiles version 1
	 TD-automobile_price-Clean_Missing_Data-Cleaning_transformation-0d2854af version 1
	 MD-automobile_price-Train_Model-Trained_model-b8acb5ad version 1
